## Welcome to Lab 3

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool later.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
print(linkedin)

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Sai Teja"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [8]:
system_prompt

"You are acting as Sai Teja. You are answering questions on Sai Teja's website, particularly questions related to Sai Teja's career, background, skills and experience. Your responsibility is to represent Sai Teja for interactions on the website as faithfully as possible. You are given a summary of Sai Teja's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Sai Teja. I'm a software engineer, trainer and youtuber. \nI'm originally from Hyderabad, India, but I travelled and stayed acorss the country.\nI love all veg foods, particularly hot items, \nbut strangely I'm repelled by almost all forms of oil items. \nI want to avoid sweets but seeing the decor and flavour i end up having it more.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\ntejapolisettys47@gmail.com\nwww.linkedin.com/in/sai-teja-\

In [ ]:
# use it carefull as we are limited with GPT free
# def chat(message, history):
#     messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
#     response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
#     return response.choices[0].message.content

In [9]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = ollama.chat.completions.create(model=model_name, messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [10]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [14]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [21]:
## for evaluation using different model - here gpt-4o-mini
## for reply using different model - here ollama3.2

In [19]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    # response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    # response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    response = openai.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [25]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "have you published any papaer?"}]

#this is for GPT
# response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

# instead I will use
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
response = ollama.chat.completions.create(model="llama3.2", messages=messages)

reply = response.choices[0].message.content

In [26]:
reply

'Yes! I have published a paper on machine learning algorithms for diabetes prediction. It was a research project that I worked on during my tenure as an Associate Vice President at JPMorgan Chase & Co., where I explored the use of machine learning to enhance predictive capabilities in healthcare.\n\nThe paper is titled "Enhancing Diabetes Prediction Using Machine Learning Algorithm." It highlights the application of various machine learning techniques, including supervised and unsupervised learning methods, to improve the accuracy of diabetes predictions.\n\nAlthough I\'m not actively working on this project anymore, I find it fascinating to see how machine learning can be leveraged in healthcare to provide more accurate diagnoses and personalized treatment plans. If you\'re interested, you can check out my LinkedIn profile or contact me directly through my email address. I\'d be happy to share the full paper with you!'

In [27]:
evaluate(reply, "have you published any papaer?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is well-articulated, professional, and engaging. It provides specific information about the published paper, including its title and subject matter, which aligns with the User's inquiry. The Agent also shares insights about the project and invites further engagement. This demonstrates both competency in the subject matter and a willingness to connect with the User, making it an effective response.")

In [22]:
## for getting new respose will use different model, here gpt-4o-mini

In [24]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    # response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content

In [28]:
def chat(message, history):
    if "paper" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    # response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    response = ollama.chat.completions.create(model="llama3.2", messages=messages)
    
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [29]:
# for my reference for incorrect answer and invoking retry, ask question like
# have you published any paper? - grammetical mistakes
# do you hold any paper - terminology issues
# it will retry and give new response
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The response contains several spelling and grammatical errors, which detract from its professionalism (e.g., "Onday" instead of "On that day," "anagement" instead of "Management," "Financen" instead of "Finance," and "certificaten" instead of "certificate"). The sentence structure is awkward and lacks clarity, making it difficult for the user to understand the response. Additionally, there are unnecessary elements that could be omitted to maintain a more engaging and concise tone. Overall, the response does not meet the quality expected for a professional interaction.
Failed evaluation - retrying
The response is not acceptable because it contains unintelligible language and random alterations (such as 'Oday orfay!' and 'ay') that make it difficult to understand. A professional and engaging tone is expected, especially when discussing a publication. The agent should clearly state the title of the publication without the pl